# Integration with a hardware synthesis tool for ML

In [1]:
import os
import numpy as np
import tensorflow as tf 
from tensorflow.keras.models import *
from tensorflow.keras.layers import *
from qkeras import *
from qkeras import QActivation
from qkeras import QDense, QConv2DBatchnorm
import hls4ml

2023-09-27 20:16:18.075136: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [2]:
# Path Vivado HLS 
os.environ['PATH'] = '/tools/Xilinx2019/Vivado/2019.2/bin:' + os.environ['PATH']
os.environ['PATH']


'/tools/Xilinx2019/Vivado/2019.2/bin:/tools/anaconda3/envs/neuralEnv/bin:/home/ro/.local/bin:/usr/local/cuda-11.0/bin:/tools/anaconda3/envs/neuralEnv/bin:/tools/anaconda3/condabin:/home/ro/.local/bin:/usr/local/sbin:/usr/local/bin:/usr/sbin:/usr/bin:/sbin:/bin:/usr/games:/usr/local/games:/snap/bin'

In [3]:
# Load keras model 

#distilled_student_1D

model = load_model('../compressionAndTraining/models/distilled_student_2D.h5', custom_objects={'QConv2DBatchnorm':QConv2DBatchnorm,'quantized_bits':quantized_bits, 'QActivation': QActivation, 'QDense': QDense})
model.summary()

model.summary()


2023-09-27 20:16:19.860676: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2023-09-27 20:16:19.861346: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2023-09-27 20:16:20.254114: E tensorflow/stream_executor/cuda/cuda_driver.cc:328] failed call to cuInit: CUDA_ERROR_UNKNOWN: unknown error
2023-09-27 20:16:20.254139: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:169] retrieving CUDA diagnostic information for host: mareKaleido
2023-09-27 20:16:20.254143: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:176] hostname: mareKaleido
2023-09-27 20:16:20.254293: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:200] libcuda reported version is: 515.43.4
2023-09-27 20:16:20.254305: I tensorflow/stream_executor/cuda/cuda_diagnostics.cc:204] kernel reported version is: 515.43.4
2023-09-27 20:16:20.254308: I tensorflow/stream_executor/cuda/cuda_diagnostics

Model: "qkeras"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 80, 80, 3)]       0         
_________________________________________________________________
conv1 (QConv2DBatchnorm)     (None, 80, 80, 2)         65        
_________________________________________________________________
relu1 (QActivation)          (None, 80, 80, 2)         0         
_________________________________________________________________
conv2 (QConv2DBatchnorm)     (None, 80, 80, 7)         162       
_________________________________________________________________
relu2 (QActivation)          (None, 80, 80, 7)         0         
_________________________________________________________________
pool_0 (MaxPooling2D)        (None, 40, 40, 7)         0         
_________________________________________________________________
conv3 (QConv2DBatchnorm)     (None, 40, 40, 3)         205  

## hls4ml 

In [4]:

# Based on the tutorials provided by hls4ml

hls4ml.model.optimizer.OutputRoundingSaturationMode.layers = ['Activation']
hls4ml.model.optimizer.OutputRoundingSaturationMode.rounding_mode = 'AP_RND'
hls4ml.model.optimizer.OutputRoundingSaturationMode.saturation_mode = 'AP_SAT'

hls_config = hls4ml.utils.config_from_keras_model(model, granularity='name')
hls_config['Model']['Precision'] = 'ap_fixed<8,4>'

for Layer in hls_config['LayerName'].keys():
    hls_config['LayerName'][Layer]['Strategy'] = 'Latency'
    hls_config['LayerName'][Layer]['ReuseFactor'] = 1

#hls_config['LayerName']['output_softmax']['Strategy'] = 'Stable'

cfg = hls4ml.converters.create_config(backend='Vivado')
cfg['IOType']     = 'io_stream' # Must set this if using CNNs!
cfg['HLSConfig']  = hls_config
cfg['KerasModel'] = model
cfg['OutputDir']  = 'model_mobileNet/'
cfg['XilinxPart'] = 'xczu9eg-ffvb1156-2-e'  # PYNQ-Z1 or Zedboard: xc7z020-clg484-1
  
hls_model = hls4ml.converters.keras_to_hls(cfg)
hls_model.compile()

hls_model.build(csim=False, export=True)

Interpreting Model
Topology:
Layer name: input_1, layer type: Input
Layer name: conv1, layer type: QConv2DBatchnorm
Layer name: relu1, layer type: QActivation
Layer name: conv2, layer type: QConv2DBatchnorm
Layer name: relu2, layer type: QActivation
Layer name: pool_0, layer type: MaxPooling2D
Layer name: conv3, layer type: QConv2DBatchnorm
Layer name: relu3, layer type: QActivation
Layer name: conv4, layer type: QConv2DBatchnorm
Layer name: relu4, layer type: QActivation
Layer name: pool_1, layer type: MaxPooling2D
Layer name: output, layer type: QDense
Layer name: softmax, layer type: Activation
Interpreting Model
Topology:
Layer name: input_1, layer type: InputLayer, input shapes: [[None, 80, 80, 3]], output shape: [None, 80, 80, 3]
Layer name: conv1, layer type: QConv2DBatchnorm, input shapes: [[None, 80, 80, 3]], output shape: [None, 80, 80, 2]
Layer name: relu1, layer type: Activation, input shapes: [[None, 80, 80, 2]], output shape: [None, 80, 80, 2]
Layer name: conv2, layer typ